In [1]:
import pandas as pd

In [ ]:

dfe = pd.read_excel('../data/документация/КТ_Кировский.xlsx')
dfc = pd.read_csv('../data/документация/sp.csv')


#dfe.merge(dfc)


,НомерДокумента,Номер,Счет,Сумма,Пени,ГосПошлина
0,9-84-16/2024,2148590262,1561382,"14510,84","172,55","293,67"
1,2-103-4262/2024,2148781880,1056981,"3726,63","164,12",2000
2,2-107-2433/2024,2148706711,4167749,"13633,65","85,85","274,54"
3,2-107-4312/2024,2148770161,3746518,"4971,07","132,07",200
4,2-107-4312/2024,2148770161,3746518,"4971,07","264,14",200
...,...,...,...,...,...,...
59610,СП 2-14-908/2020,2147682167,1359619,"16152,36",0,"216,54"
59611,СП № 2-81-2463/2022,2148202053,860143,"18247,34","1641,63","397,78"
59612,СП2-145-1508/2024,2148636092,238089,"5687,36","4787,98","217,82"
59613,СП2-145-1508/2024,2148636092,238089,"5687,36","5203,5","217,82"
